<a href="https://colab.research.google.com/github/niravrupapara/Pytorch/blob/main/RNN_pytorch_simple_QA_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gdown

file_id = "1X4Hcj72NK7J2JYvgjICFj0R1XwUq1w0a"
url = f"https://drive.google.com/uc?export=download&id={file_id}"

gdown.download(url, "100_Unique_QA_Dataset.csv", quiet=False)


Downloading...
From: https://drive.google.com/uc?export=download&id=1X4Hcj72NK7J2JYvgjICFj0R1XwUq1w0a
To: /content/100_Unique_QA_Dataset.csv
100%|██████████| 4.28k/4.28k [00:00<00:00, 13.0MB/s]


'100_Unique_QA_Dataset.csv'

In [2]:
import pandas as pd
df = pd.read_csv("100_Unique_QA_Dataset.csv")


In [3]:
##fun
def tokenize(text):
  text = text.lower()
  text = text.replace("?","")
  text = text.replace("'","")

  return text.split()

In [4]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [5]:
# vocab
vocab = {'<UNK>':0}

In [6]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:
    if token not in vocab:
      vocab[token] = len(vocab)

In [7]:
df.apply(build_vocab, axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [8]:
len(vocab)

324

In [9]:
# convert words to numerical indices
def text_to_indices(text, vocab):
  indexed_text=[]

  for token in tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text


In [10]:
text_to_indices("What is campusx", vocab)

[1, 2, 0]

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader

In [15]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)

    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [16]:
dataset = QADataset(df, vocab)

In [19]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [20]:
for question, answer in dataloader:
  print(question, answer[0])

tensor([[  1,   2,   3, 103,   5, 104,  19, 105]]) tensor([106])
tensor([[ 42, 137, 118,   3, 247,   5, 248]]) tensor([249])
tensor([[ 1,  2,  3, 69,  5, 53]]) tensor([260])
tensor([[ 42, 137,   2,  62,  39,   3, 322, 323]]) tensor([6])
tensor([[ 1,  2,  3, 17, 18, 19, 20, 21, 22]]) tensor([23])
tensor([[  1,   2,   3,   4,   5, 109]]) tensor([317])
tensor([[ 1,  2,  3,  4,  5, 73]]) tensor([74])
tensor([[ 42, 101,   2,   3,  17]]) tensor([102])
tensor([[ 10,   2,  62,  63,   3, 283,   5, 284]]) tensor([285])
tensor([[ 1,  2,  3, 33, 34,  5, 35]]) tensor([36])
tensor([[ 1,  2,  3, 92, 93, 94]]) tensor([95])
tensor([[ 42,   2,   3, 274, 211, 275]]) tensor([276])
tensor([[ 10,  29, 130, 131]]) tensor([132])
tensor([[  1,   2,   3,   4,   5, 135]]) tensor([136])
tensor([[ 78,  79, 261, 151,  14, 262, 153]]) tensor([36])
tensor([[ 10, 140,   3, 141, 270,  93, 271,   5,   3, 272]]) tensor([273])
tensor([[10, 29,  3, 30, 31]]) tensor([32])
tensor([[ 1,  2,  3,  4,  5, 53]]) tensor([54])
tens

In [21]:
import torch.nn as nn

In [32]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()

    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(input_size=50, hidden_size=64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden , final = self.rnn(embedded_question)
    output = self.fc(final)
    output = self.fc(final.squeeze(0))


    return output


In [33]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [34]:
learning_rate = 0.001
epochs = 20

In [35]:
model = SimpleRNN(len(vocab))

In [36]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [37]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 525.565933
Epoch: 2, Loss: 457.002537
Epoch: 3, Loss: 377.584800
Epoch: 4, Loss: 317.603402
Epoch: 5, Loss: 266.438113
Epoch: 6, Loss: 217.797114
Epoch: 7, Loss: 172.471603
Epoch: 8, Loss: 133.533583
Epoch: 9, Loss: 102.036056
Epoch: 10, Loss: 76.358425
Epoch: 11, Loss: 58.412968
Epoch: 12, Loss: 45.556859
Epoch: 13, Loss: 35.562733
Epoch: 14, Loss: 28.775963
Epoch: 15, Loss: 23.631834
Epoch: 16, Loss: 19.606063
Epoch: 17, Loss: 16.422150
Epoch: 18, Loss: 14.048226
Epoch: 19, Loss: 12.147809
Epoch: 20, Loss: 10.486415


In [38]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [39]:
predict(model, "What is the largest planet in our solar system?")

jupiter
